# Embassies
Get all of the addresses of the embassies.

In [1]:
import bs4
import requests
import pandas as pd

In [2]:
with open('countries.txt') as o:
    countries = o.read().splitlines()

In [3]:
url = 'https://embassy.goabroad.com/embassies-in/india'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, 'lxml')

In [4]:
def parse_info(info):
    infos = {}
    for name, *details in info.find_all('div', {'class': None}): # only non-"info-row"
        name = str(name.string)
        text = parsers[name](details)
        infos[name] = text
    return infos

def parse_title(title, office_types=['Embassy', 'Consulate']):
    """Return Nationality and Office Type parse from title"""
    title = title.split()
    office = set(title).intersection(office_types).pop()
    nationality = ' '.join(title[:title.index(office)])
    return nationality, office
        
def parse_simple_details(details):
    string, = details
    return string.strip()

def parse_details_with_breaks(details):
    details = [detail.strip() for detail in details
               if type(detail) == bs4.element.NavigableString]
    return ', '.join(details)

def parse_details_with_links(details):
    details = [detail.text for detail in details if detail.name == 'a']
    return ','.join(details)

parsers = {'City': parse_simple_details, 'Phone': parse_details_with_breaks,
           'Fax': parse_details_with_breaks, 'Details': parse_details_with_breaks,
           'Office Hours': parse_details_with_breaks, 'Email': parse_details_with_links,
           'Website': parse_details_with_links}

In [5]:
embassies_info = {}
embassies = soup.find('section', {'class': 'search-embassies program-listing'})
for embassy in embassies.find_all('div', {'class': 'body'}):
    embassy_info = {}
    title = str(embassy.h6.string)
    # Parse embassy information
    embassy_info.update(parse_info(embassy.find('section', {'class': 'embassy-info'})))
    embassy_info['Nationality'], embassy_info['Office Type'] = parse_title(title)
    embassy_info['Address'] = embassy.find('span', 'embassy-address').string.strip()
    
    embassies_info[title] = embassy_info

df = pd.DataFrame(embassies_info).transpose()

In [6]:
df

,Address,City,Details,Email,Fax,Nationality,Office Hours,Office Type,Phone,Website
"Afghani Consulate in Mumbai, India","115, Walkeshwar Road - 400006, India",Mumbai,Consul General: Mr. Mohammad Aman Amin,afghancg_mumbai@yahoo.com,(+91-22) 23635437,Afghani,NaN,Consulate,(+91-22) 23633777,NaN
"Afghani Embassy in New Delhi, India","Plat No. 5, Block 50F, Shantipath, Chanakyapur...",New Delhi,Ambassador: H.E Shaida Mohammad Abdali,afghanspirit@yahoo.com,(+ 91-112) 687 5439,Afghani,NaN,Embassy,(+91-112) 410 331,NaN
Albanian Embassy in India,"B 2, West End, New Delhi, DL 110021, India",NaN,"Mr Fatos Kerciku, Ambassador",embassy.delhi@mfa.gov.al,+91-11-4610-8285,Albanian,Monday to Friday\r\n09:00 - 17:00,Embassy,+ 91-11-4059-1294,http://www.ambasadat.gov.al/india/en/
Algerian Embassy in India,"2/2, Shanti Niketan, New Delhi 110021, India",NaN,"Mr Mohammed-Hacene Echarif, Ambassador",embalgindia@hotmail.com,(+91) 11-2411 7590,Algerian,09.00-16.00,Embassy,(+91) 11-2411 7585 / 6 / 8,http://www.embalgindia.com
"American Consulate in Chennai, India","Gemini Circle, Chennai 600006, India",Chennai,NaN,chennaic@state.gov,044-2811-2020,American,NaN,Consulate,044-2857-4000,http://chennai.usconsulate.gov
"American Consulate in Hyderabad, India","Paigah Palace, 1-8-323, Chiran Fort Lane, Begu...",Hyderabad,NaN,HyderabadPA@state.gov,NaN,American,The Consulate General of the United States of ...,Consulate,040-4625-8222 / 0120-484-4644,http://hyderabad.usconsulate.gov
"American Consulate in Kolkata, India","5/1 Ho Chi Minh Sarani, Kolkata 700071, India",Kolkata,NaN,kolkataPAS@state.gov,91-33-2288-1616/0356,American,"0800 - 1300, 1400 - 1700, (Monday through Friday)",Consulate,91-33-3984-6300,http://kolkata.usconsulate.gov
"American Consulate in Mumbai, India","C-49, G-Block, Bandra Kurla Complex, Bandra Ea...",Mumbai,NaN,mumbaipublicaffairs@state.gov,NaN,American,NaN,Consulate,011-91-22-2672-4000,http://mumbai.usconsulate.gov
"American Embassy in New Delhi, India","Shantipath, Chanakyapuri, New Delhi - 110021",New Delhi,NaN,NDwebmail@state.gov,011-91-11-2419-0017,American,NaN,Embassy,011-91-11-2419-8000,http://newdelhi.usembassy.gov
Angolan Consulate in India,"141 Atlanta, 14th Floor Nariman Point , Mumbai...",NaN,Mr. Sharma Nand Sharma - Consul General,maria@bom3.vsnl.net.in,(+91) 22-2287 5467,Angolan,10.00-17.00,Consulate,(+91) 22-2285 1430,NaN
